# Main Historical Extraction Engine
Run as many times as needed to get full list of players. 

Warning: Full extract takes >3 hours

In [2]:
# import python libraries and nba library
import pandas as np
import nba_api as nba
from matplotlib import pyplot as plt
import pandas as pd
import socket
import logging
import os
import glob

In [3]:
# Get all files in working data directory
path = os.getcwd()
path = path.replace("notebooks\\basketballProject\\extraction","data\\basketballProject")+'\\'+'historical-extraction-files\\'
csv_files = glob.glob(os.path.join(path,"*.csv"))
write_path = "C:/Github Repo/NBA-Analytics/data/basketballProject/historical-extraction-files/nba_data_"
# write_path = "../../data/basketballProject/historical-extraction-files/nba_data_"

In [4]:
# Check what players you have already analyzed to avoid duplication
check_df = pd.DataFrame()
for file in csv_files:

    df = pd.read_csv(file)
    frames = [check_df, df]
    check_df = pd.concat(frames)

In [5]:
try:
    unique_ids = check_df.PLAYER_ID.unique()
except:
    pass

In [6]:
# import teams, players, and sort them into active or all players
from nba_api.stats.static import teams, players
nba_players = players.get_players()
# active_players = players.get_active_players()
teams = teams.get_teams()

In [7]:
# import use-case specific functions
from nba_api.stats.endpoints import playercareerstats

In [8]:
# Add entry for each player on whether they have been analyzed or not. 
# Default is not analyzed
for player in nba_players:
    player['Parsed ?'] = False
    try:
        if player["id"] in unique_ids:
            player["Parsed ?"] = True
    except:
        pass

In [9]:
# Initialize collection dataframe
master_list = pd.DataFrame()

# Create temp dataframe and initialize count to serve as a staging 
# Dataframe and break up the merging into small batches
temp_df = pd.DataFrame()
count = 0

In [10]:
# Parse thru each active player
first_id = 0
last_id = 0
try:
    for player in nba_players:

        # Check if they have been already analyzed
        if player['Parsed ?'] == True:
            pass
        
        # If they haven't, proceed
        else:
            passedId = player['id']
            player_name = player["full_name"]
            # Get player stats based on their ID
            career_stats = playercareerstats.PlayerCareerStats(player_id = passedId).get_data_frames()[0]
            
            # Get ID of first and last players analyzed for file naming purposes
            if first_id == 0:
                first_id = passedId
            last_id = passedId

            # Add player names to resulting dataframe to keep track of players
            career_stats.insert(1,"full_name",player_name)

            # Append resulting dataframe to temp dataframe
            frames = [temp_df,career_stats]
            temp_df = pd.concat(frames)

            # Set analyzed flag to true
            player['Parsed ?'] = True
            count = count + 1
            print(player_name)
            # Once enough volume is reached in the temp dataframe, append it to the main and re-initialize the dataframe
            if (count%20) == 0:
                frames = [master_list, temp_df]
                master_list = pd.concat(frames)
                temp_df = pd.DataFrame()
                print("Temp Full - Resetting")
except:
    frames = [master_list, temp_df]
    master_list = pd.concat(frames)
    temp_df = pd.DataFrame()
    print("Temp Full - Resetting")
    print("TimeoutError")

Alaa Abdelnaby
Zaid Abdul-Aziz
Kareem Abdul-Jabbar
Mahmoud Abdul-Rauf
Tariq Abdul-Wahad
Shareef Abdur-Rahim
Tom Abernethy
Forest Able
John Abramovic
Alex Abrines
Precious Achiuwa
Alex Acker
Donald Ackerman
Mark Acres
Charles Acton
Quincy Acy
Alvan Adams
Don Adams
Hassan Adams
Jaylen Adams
Temp Full - Resetting
Jordan Adams
Michael Adams
Steven Adams
Rafael Addison
Bam Adebayo
Deng Adel
Rick Adelman
Jeff Adrien
Arron Afflalo
Ochai Agbaji
Maurice Ager
Mark Aguirre
Blake Ahearn
Danny Ainge
Alexis Ajinca
Henry Akin
Josh Akognon
DeVaughn Akoon-Purcell
Solomon Alabi
Mark Alarie
Temp Full - Resetting
Gary Alcorn
Santi Aldama
Furkan Aldemir
Cole Aldrich
LaMarcus Aldridge
Chuck Aleksinas
Cliff Alexander
Cory Alexander
Courtney Alexander
Gary Alexander
Joe Alexander
Kyle Alexander
Ty-Shon Alexander
Victor Alexander
Nickeil Alexander-Walker
Steve Alford
Rawle Alkins
Bob Allen
Grayson Allen
Jarrett Allen
Temp Full - Resetting
Jerome Allen
Kadeem Allen
Lavoy Allen
Lucius Allen
Malik Allen
Randy All

In [11]:
if first_id == 0:
    print("empty")
    pass
else:
    master_list.to_csv(write_path+str(first_id)+"to"+str(last_id)+".csv")